<a href="https://colab.research.google.com/github/atul-ai/IndicHistoryExperiments/blob/main/SimpleNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Named Entity Recognition (NER)

We will run  NER on the Shivaji book. We will do this with IndicNER model from AI4Bharat. The goal is to test the quality of the NER on the Maratha History books using this model.

This is part of the larger project to build knowledge graph of the IndicHistory and eventually build a IndicHistory chat bot to answer knowledged backed questions about Indic History. You can learn about details of this project at <Link Of the Project>

In [ ]:
!pip3 install transformers datasets sentencepiece seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=34ffb884bec408243c04eee2e6cc4fb22e5b6fc3df267fb43fde0b800409f5d1
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")
model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

In [ ]:
def get_predictions( sentence, tokenizer, model ):
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    logits = model(**tok_sentence).logits.argmax(-1)
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]

    predicted_labels = []
    previous_token_id = 0
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

    return predicted_labels

In [ ]:
sentence = 'लगातार हमलावर हो रहे शिवपाल और राजभर को सपा की दो टूक, चिट्ठी जारी कर कहा- जहां जाना चाहें जा सकते हैं'
predicted_labels = get_predictions(sentence=sentence, tokenizer=tokenizer, model=model)
words = sentence.split(' ')
for index in range(len(words)):
  print( words[index] + '\t' + predicted_labels[index] )

लगातार	O
हमलावर	O
हो	O
रहे	O
शिवपाल	B-PER
और	O
राजभर	B-PER
को	O
सपा	B-ORG
की	O
दो	O
टूक,	O
चिट्ठी	O
जारी	O
कर	O
कहा-	O
जहां	O
जाना	O
चाहें	O
जा	O
सकते	O
हैं	O


In [ ]:
# prompt: write code to read the file from the data folder and split on paragraphs and create an iterator to call the get_prediction function with it.
import re
def process_file(file_path):
  with open(file_path, 'r') as f:
    content = f.read()
  paragraphs = content.split('\n\n')

  for paragraph in paragraphs:
    if paragraph.strip():
      # Split paragraph into sentences using regex
      sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', paragraph)
      for sentence in sentences:
        yield sentence.strip()

# Example usage (assuming your file is in 'data' folder)
file_path = '/content/data/Chapter_1.txt'  # Replace with the actual file path
for sentence in process_file(file_path):
  predicted_labels = get_predictions(sentence=sentence, tokenizer=tokenizer, model=model)
  # Do something with the predicted_labels, e.g., print them
  words = sentence.split(' ')
  for index in range(len(words)):
    print( words[index] + '\t' + predicted_labels[index] )

Chapter	O
1
India	O
in	B-LOC
Shivaji’s	O
Times
PROMINENT	B-LOC
MUSLIM	O
ST	O
ATES	O
IN	O
INDIA
Muhammad,	O
the	O
founder	O
of	O
Islam,	O
died	B-PER
in	O
632	O
and	O
was
succeeded	O
by	B-LOC
a	O
line	O
of	O
Caliphs,
1	O
who	O
were	O
both	O
religious	O
and
political	O
heads	O
of	O
the	O
Muslim	O
world.	O
Under	O
their	O
rule,	O
the	O
Arabs,
red	O
by	O
the	O
spirit	O
of	O
the	O
Jihad,	O
began	O
to	O
advance	O
in	O
all
directions.	O
In	O
the	O
west,	O
their	O
wave	O
of	O
conquest	O
rolled	O
along	O
the
northern	O
coast	O
of	O
Africa,	O
reached	I-LOC
the	I-LOC
Atlantic	O
in	O
710	O
and
crossed	O
into	O
Spain	O
in	O
the	O
following	O
year.	B-LOC
After	O
establishing	O
their
hold	O
in	O
the	O
Iberian	O
Peninsula,	B-LOC
they	I-LOC
crossed	O
the	O
Pyrenees	O
and
entered	O
France.	B-LOC
There,	O
in	O
732,	O
Charles	O
Martel,	O
the	B-PER
commander-
in-chief	I-PER
and	O
the	O
de	O
facto	O
king	O
of	O
the	O
Franks,	O
routed	O
them	O
at
Tours,	O
some	O
320	O
km	O
south	O
of	O
Paris.	O


KeyboardInterrupt: 